### Capstone 1 - Washington state linkage of infant death, birth, and mother's hospitalization discharge data

##### Maya Bhat-Gregerson

January 7, 2020

### B. PREPARATION OF DEATH DATA 2016-2018

### I. Data acquisition

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

I use SQL queries to get the death and birth variables I am likely to need for linking the records. 

NOTE: I renamed all variables from death records so that they begin with 'd' to distinguish the fields from those in birth records with the same names.

**DEATH DATA 2016, 2017, 2018**

In [10]:
## CONNECT TO WHALES & USE SQL QUERY FOR DEATH DATA SET

driver = '{SQL Server Native Client 11.0}'

conn = pyodbc.connect(
        Trusted_Connection='Yes',
        Driver='{ODBC Driver 13 for SQL Server}',
        Server='####',
        Database='####'
        )

querystring = ("SELECT SFN_NUM as 'dsfn'," + 
        "BIRTH_SFN_NUM as 'dbirsfn', " + 
        "SSN as 'dssn', " +
        "ISNULL(GNAME, 'NaN') as 'dfname', " +
        "ISNULL(MNAME, 'NaN') as 'dmname', " +
        "ISNULL(LNAME, 'NaN') as 'dlname', " +
        "ISNULL(MOTHER_GNAME, 'NaN') as 'dmom_fname', " +
        "ISNULL(MOTHER_MNAME, 'NaN') as 'dmom_mname', " + 
        "ISNULL(MOTHER_LNAME, 'NaN') as 'dmom_lname', " +
        "ISNULL(SEX, 'NaN') as 'dsex', " + 
		"AGETYPE as 'dagetype', " +
		"CONCAT(AGE1, AGE2, AGE3, AGE4, AGE5) as 'dage', " + 
		"ISNULL(AGE1_CALC, '999') as 'dageyrs'," + 
		"DOB as 'ddob', " + 
		"ISNULL(SUBSTRING(DOB, 1,2), '99') as 'ddobm', " + 
		"ISNULL(SUBSTRING(DOB, 4,2), '99') as 'ddobd', " +
		"ISNULL(SUBSTRING(DOB, 7,4), '9999') as 'ddoby', " + 
		"ISNULL(DOD, '  ') as 'ddod'," + 
		"ISNULL(SUBSTRING(DOD, 1,2), '99') as 'ddodm', " + 
		"ISNULL(SUBSTRING(DOD, 4,2), '99') as 'ddodd', " + 
		"ISNULL(SUBSTRING(DOD, 7,4), '9999') as 'ddody', " +
		"ISNULL(DSTATEL, '  ') as 'ddthstatel', " +
		"ISNULL(BPLACE_ST_FIPS_CD, '  ') as 'dbirplstatefips', " + 
		"ISNULL(BPLACE_CNT, '  ') as 'dbircountryl'," +
        "ISNULL(BPLACE_CNT_FIPS_CD, ' ') as 'dbircountryfips', " +
		"ISNULL(DNAME_CITY, '  ') as 'ddthcityl', " +
        "ISNULL(DNAME_FIPS_CD, ' ') as 'ddthcityfips'," +
		"ISNULL(DCOUNTY, '  ') as 'ddthcountyl'," +
        "ISNULL(DCOUNTY_FIPS_CD, ' ') as 'ddthcntyfips'," +    
		"ISNULL(SUBSTRING(DZIP9, 1,5), '99999') as 'ddthzip', " + 
		"RIGHT('000' + ISNULL(DFACILITYL, '  '), 3) as 'ddfaccode'," + 
		"ISNULL(RACE_NCHS_CD, '99') as 'dbridgerace', " + 
		"ISNULL(DETHNIC_NO, '  ') as 'dhispno', " + 
		"ISNULL(RES_CITY, '  ') as 'drescity', " + 
		"RIGHT('00000' + ISNULL(RES_CITY_FIPS_CD, '99999'), 5) as 'drescityfips', " + 
		"ISNULL(RES_LIMITS, '  ') as 'drescitylim', " + 
		"ISNULL(RES_COUNTY, '  ') as 'drescountyl', " + 
		"RIGHT('000' + ISNULL(RES_COUNTY_FIPS_CD, '999'), 3) as 'drescntyfips', " + 
		"ISNULL(RES_STATE_FIPS_CD, '  ') as 'dresstatefips', " + 
		"ISNULL(SUBSTRING(RES_ZIP, 1,5), '99999') as 'dreszip'" + 
"FROM [wa_vrvweb_events].[VRV_DEATH_TBL]" +
"WHERE ((DATE_DEATH_YEAR = 2016) OR (DATE_DEATH_YEAR = 2017) OR (DATE_DEATH_YEAR = 2018))" +
	"AND VRV_REGISTERED_FLAG = '1'" +
	"AND FL_CURRENT = '1'" +
	"AND FL_VOIDED = '0'")

dth1618= pd.read_sql_query(querystring, conn)

## SAVE DATA AS CSV FILE

dth1618.to_csv(r'####\Py\Data\dth1618_raw.csv', index=None, header=True)

In [11]:
d1618 = pd.read_csv(r'####\Py\Data\dth1618_raw.csv', low_memory=False)

In [12]:
d1618.shape

(172809, 40)

In [13]:
d1618.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172809 entries, 0 to 172808
Data columns (total 40 columns):
dsfn               172809 non-null int64
dbirsfn            7507 non-null object
dssn               172809 non-null object
dfname             172807 non-null object
dmname             157996 non-null object
dlname             172802 non-null object
dmom_fname         172746 non-null object
dmom_mname         75899 non-null object
dmom_lname         172750 non-null object
dsex               172808 non-null object
dagetype           172809 non-null float64
dage               172806 non-null float64
dageyrs            172809 non-null float64
ddob               172809 non-null object
ddobm              172809 non-null int64
ddobd              172809 non-null int64
ddoby              172809 non-null int64
ddod               172809 non-null object
ddodm              172809 non-null int64
ddodd              172809 non-null int64
ddody              172809 non-null int64
ddthstatel    

In [14]:
dthlinkvars = list(d1618.columns.values)
dthlinkvars

['dsfn',
 'dbirsfn',
 'dssn',
 'dfname',
 'dmname',
 'dlname',
 'dmom_fname',
 'dmom_mname',
 'dmom_lname',
 'dsex',
 'dagetype',
 'dage',
 'dageyrs',
 'ddob',
 'ddobm',
 'ddobd',
 'ddoby',
 'ddod',
 'ddodm',
 'ddodd',
 'ddody',
 'ddthstatel',
 'dbirplstatefips',
 'dbircountryl',
 'dbircountryfips',
 'ddthcityl',
 'ddthcityfips',
 'ddthcountyl',
 'ddthcntyfips',
 'ddthzip',
 'ddfaccode',
 'dbridgerace',
 'dhispno',
 'drescity',
 'drescityfips',
 'drescitylim',
 'drescountyl',
 'drescntyfips',
 'dresstatefips',
 'dreszip']

### II. Data cleaning and standardization

In [68]:
# look at data
#d1618.head()

  - Check death record number 'dsfn' and convert to integer if necessary

In [16]:
d1618.dsfn.describe()

count    1.728090e+05
mean     2.017043e+09
std      8.144065e+05
min      2.016000e+09
25%      2.016043e+09
50%      2.017031e+09
75%      2.018015e+09
max      2.018091e+09
Name: dsfn, dtype: float64

In [17]:
dth1618['dsfn'] = dth1618['dsfn'].astype(int)

In [18]:
dth1618.dsfn.dtype

dtype('int32')

**RESTRICT TO WA RESIDENTS**

In [22]:
d1618['dresstatefips'].value_counts(dropna=False)

WA    168702
OR      1424
ID       605
CA       311
AK       308
MT       187
XX       139
ZZ       131
AZ       117
TX        82
FL        73
BC        52
HI        46
NV        45
NY        42
IL        38
MI        36
CO        30
UT        30
PA        24
MN        23
OH        19
OK        18
NJ        18
VA        18
IN        17
MO        15
TN        15
AL        14
IA        13
SC        13
SD        12
LA        12
MD        12
NC        12
WI        11
KS        11
NM        11
MA        11
GA        10
AR        10
MS         9
ND         9
WY         9
CT         8
AB         7
NE         7
ME         5
GU         4
ON         4
KY         4
DE         4
NH         4
WV         4
AS         3
DC         2
RI         2
PR         2
VT         1
NS         1
MP         1
YT         1
SK         1
Name: dresstatefips, dtype: int64

In [24]:
# keep only WA residents and check if successful in filtering out other state residents.
d1618 = d1618[(d1618['dresstatefips']=="WA")]
d1618['dresstatefips'].value_counts(dropna=False)

WA    168702
Name: dresstatefips, dtype: int64

#### LIMIT TO INFANTS
Infants are defined as less than 365 days old at the time of death.

Variable 'dagetype' indicates the unit of time of the value in 'dage' - years, months, weeks, days, minutes etc.  Select all except years to keep infant deaths.

In [25]:
d1618['dagetype'].value_counts(dropna=False)

1.0    167566
2.0       405
4.0       285
3.0       283
5.0       159
6.0         2
9.0         2
Name: dagetype, dtype: int64

In [27]:
d1618 = d1618[(d1618.dagetype != 1)]
d1618.dagetype.value_counts(dropna=False)

2.0    405
4.0    285
3.0    283
5.0    159
6.0      2
9.0      2
Name: dagetype, dtype: int64

In [28]:
d1618.shape

(1136, 40)

#### CHECK FOR NULL VALUES

In [29]:
d1618.isna().sum()

dsfn                 0
dbirsfn              3
dssn                 0
dfname               0
dmname             183
dlname               0
dmom_fname           0
dmom_mname         496
dmom_lname           0
dsex                 0
dagetype             0
dage                 2
dageyrs              0
ddob                 0
ddobm                0
ddobd                0
ddoby                0
ddod                 0
ddodm                0
ddodd                0
ddody                0
ddthstatel           0
dbirplstatefips      0
dbircountryl         0
dbircountryfips      0
ddthcityl            0
ddthcityfips         0
ddthcountyl          0
ddthcntyfips         0
ddthzip              0
ddfaccode            0
dbridgerace          0
dhispno              0
drescity             0
drescityfips         0
drescitylim          0
drescountyl          0
drescntyfips         0
dresstatefips        0
dreszip              0
dtype: int64

Probably will not use mothers' and infants' middle name for linking as there are too many missing values.

#### STANDARDIZE ALL STRING VARIABLES

First, middle, and last names of infants and mothers as well as city names will be standardized by converting these columns to upper case text, removing white spaces, removing hyphens and other punctuation marks.

In [30]:
# convert to upper case
d1618 = d1618.apply(lambda x: x.str.upper() if type(x) == str else x)

In [31]:
#remove white spaces, punctuation
d1618 = d1618.apply(lambda x: x.str.strip() if type(x) == str else x)
d1618 = d1618.applymap(lambda x: x.replace(" ", "") if type(x) == str else x)
d1618 = d1618.applymap(lambda x: x.replace("-", "") if type(x) == str else x)
d1618 = d1618.applymap(lambda x: x.replace(".", "") if type(x) == str else x)

In [41]:
#d1618.head(30)

- Verified that all string transformations were successful.

#### CHECK FOR OUT OF RANGE VALUES

In [42]:
#create dictionary of valid values so that each variable can be checked to make sure there is no
# out of range value.

valids = {'sex': ['M', 'F', 'U'],
          'dobm': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 99],
          'dobd': np.r_[1:32 ,99],
          'doby': [2016,2017,2018],
         'rcntyfips': np.r_[range(1, 78, 2), 99],
         'certtype': ['R'],
         'birthstatefips': ['WA'], 
         'rstatefips': ['WA']}

In [44]:
# check for out of range values for 'sex'

chksex = d1618['dsex'].isin(valids['sex'])
len(d1618[~chksex])

0

In [45]:
# check for out of range values for 'dobm'

chkdobm = d1618['ddobm'].isin(valids['dobm'])
len(d1618[~chkdobm])

0

In [46]:
# check for out of range values for 'doby'

chkdoby = d1618['ddoby'].isin(valids['doby'])
len(d1618[~chkdoby])

53

In [48]:
d1618['ddoby'].value_counts(dropna=False)

2016    402
2018    353
2017    328
2015     51
9999      2
Name: ddoby, dtype: int64

In [59]:
# remove 2015 deaths - these should not be in this data set.  The earliest deaths to 2016 births should have been in 2016.

d1618 = d1618[(d1618.ddoby != 2015)]
d1618.ddoby.value_counts(dropna=False)

2016    402
2018    353
2017    328
9999      2
Name: ddoby, dtype: int64

In [64]:

dobyerrors = d1618[~chkdoby][['ddob']]

dobyerrors

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,ddob
75737,99/99/9999
94604,99/99/9999


- unable to correct dob year without additional information on date of birth.

In [49]:
# check for out of range values for 'dobd'

chkdobd = d1618['ddobd'].isin(valids['dobd'])
len(d1618[~chkdobd])


0

In [53]:
# check for out of range values for 'drescntyfips'

chkrcounty = d1618['drescntyfips'].isin(valids['rcntyfips'])
len(d1618['drescntyfips'][~chkrcounty])

3

In [57]:
rcntyerrors = d1618[~chkrcounty][['dsfn','drescntyfips', 'drescountyl','drescity']]

rcntyerrors

,dsfn,drescntyfips,drescountyl,drescity
108723,2017052270,999,UNKNOWN,UNKNOWN
158264,2018043748,999,UNKNOWN,UNKNOWN
170415,2018057560,999,UNKNOWN,NOTAPPLICABLE


 - cannot replace with correct values without any information on residence city or county.

#### CONVERT ALL AGE VALUES TO AGE IN DAYS

In [67]:
#convert age into age in days

def agetodays(x):
    if x['dagetype']==2:
        return x['dage']*30
    elif x['dagetype']==3:
        return x['dage']
    elif x['dagetype']==4:
        return x['dage']/24
    elif x['dagetype']==5:
        return x['dage']/(60*24)
    
    
d1618['dagedays'] = np.array(d1618.apply(agetodays, axis=1)).astype(int)
d1618[['dagedays', 'dagetype', 'dage']].head(10)



,dagedays,dagetype,dage
265,150,2.0,5.0
508,17,3.0,17.0
848,90,2.0,3.0
888,60,2.0,2.0
892,0,5.0,2.0
1112,90,2.0,3.0
1169,180,2.0,6.0
1186,0,5.0,20.0
1288,0,5.0,25.0
1457,150,2.0,5.0


In [69]:
d1618.to_csv(r'Y:\DQSS\Death\MBG\Py\Data\d1618_clean.csv', index=None, header=True)